In [ ]:
import numpy as np

In [ ]:
import copy

In [ ]:
import tensorflow as tf

In [ ]:
# pipe Locations = 10
# type of damages = 4-{0,1,2,3}
condition = [0,1,0,2,1,0,1,3,2,0] #pipe condition initially at all locations
location = 7 #initial loc1tion of the robot
actions = [-1,0,1] #possible actions

**Simulator** **Function**

In [ ]:
# cost vector
#Cost = [0, 20, 50, 150]

In [ ]:
def one_time_step(curr_condition, curr_loc, a):
  cost = [0, 1, 10, 100]
  c= 0.1
  if(a == 0):
    c+=10
    c += cost[curr_condition[curr_loc]]
    curr_condition[curr_loc]=0
  elif(a == 1 and curr_loc < 9):
    curr_loc+=1
  elif(a == -1 and curr_loc > 0):
    curr_loc-=1
  for i in range(0,10):
    transitionName = [["11","12"],["22","23"]]
    transitionMatrix = [[0.9,0.1],[0.95,0.05]]
    if(curr_condition[i] == 1):
      change = np.random.choice(transitionName[0],replace=True,p=transitionMatrix[0])
      if change == "12":
        curr_condition[i]+=1
    elif(curr_condition[i] == 2):
      change = np.random.choice(transitionName[1],replace=True,p=transitionMatrix[1])
      if change == "23":
        curr_condition[i]+=1
  return curr_condition, curr_loc, c

In [ ]:
condition , location, c1 = one_time_step(condition, 6, 0)

In [ ]:
print(condition)
print(location)

[0, 1, 0, 2, 1, 0, 1, 3, 2, 0]


**Belief State Estimator**

In [ ]:
n_loc = 10 #number of locations
n_obs = 1  #number of observations
n_act = 3  #number of actions
n_states = 4 #number of states

In [ ]:
#transition probability matrix for every location
#P[s',a,s] is the probability of transitioning from state s to state s' given action a
P = np.array([[[1,0,0,0],[0,0.9,0.1,0],[0,0,0.95,0.05],[0,0,0,1]],
              [[1,0,0,0],[1,0,0,0],[1,0,0,0],[1,0,0,0]],
              [[1,0,0,0],[0,0.9,0.1,0],[0,0,0.95,0.05],[0,0,0,1]]])

In [ ]:
#observation probability matrix
#O[o,s,a] is the probability of observing o given action a and being in state s
O = np.array([[0.97,0.01,0.01,0.01],
              [0.01,0.97,0.01,0.01],
              [0.01,0.01,0.97,0.01],
              [0.01,0.01,0.01,0.97]])

In [ ]:
# initial belief state
# b[s] is the initial probability of being in state s
b = np.array([[0.8,0.2,0,0],
              [0.1,0.8,0.1,0],
              [0.8,0.2,0,0],
              [0,0.1,0.8,0.1],
              [0.1,0.8,0.1,0],
              [0.8,0.2,0,0],
              [0.1,0.8,0.1,0],
              [0,0,0.2,0.8],
              [0,0.1,0.8,0.1],
              [0.8,0.2,0,0]])

In [ ]:
# Compute the updated belief state
# input parameters b : row of belief states matrix for current location
# a is action performed at current location and o is te corresponding observation
def belief_state_estimator(b, a, o):
  b_new = np.zeros(n_states)
  for s_next in range(n_states):
    for s in range(n_states):
      b_new[s_next] += P[a+1,s,s_next] * b[s]
    b_new *= O[o,s_next]
  # Normalize the belief state
  b_new /= np.sum(b_new)
  return b_new

In [ ]:
b

array([[0.8, 0.2, 0. , 0. ],
       [0.1, 0.8, 0.1, 0. ],
       [0.8, 0.2, 0. , 0. ],
       [0. , 0.1, 0.8, 0.1],
       [0.1, 0.8, 0.1, 0. ],
       [0.8, 0.2, 0. , 0. ],
       [0.1, 0.8, 0.1, 0. ],
       [0. , 0. , 0.2, 0.8],
       [0. , 0.1, 0.8, 0.1],
       [0.8, 0.2, 0. , 0. ]])

**Feature State Estimator** [ curr_loc, damage, belief_distribution]

In [ ]:
# function to get feature state from given current condition and current location and belief state
def feature_state_estimator(curr_loc, damage, b):
  b1 = np.reshape(b,(1,40))[0]
  y = np.insert(b1,0,damage/4)
  y = np.insert(y,0,curr_loc/10)
  #y = [curr_loc, damage, b1]
  return y

In [ ]:
y1 = feature_state_estimator(7, condition[6], b)
print(y1)

[0.7  0.25 0.8  0.2  0.   0.   0.1  0.8  0.1  0.   0.8  0.2  0.   0.
 0.   0.1  0.8  0.1  0.1  0.8  0.1  0.   0.8  0.2  0.   0.   0.1  0.8
 0.1  0.   0.   0.   0.2  0.8  0.   0.1  0.8  0.1  0.8  0.2  0.   0.  ]


In [ ]:
# function to check if all locations are of zero damage
def check(condition):
  if(sum(condition)>0):
    return True
  return False

**Base Policy** We used a greedy policy as the starting base
policy, which repairs the current location if it is damaged, moves
left if out of all damaged locations, the nearest one is on the
left, or moves right otherwise

In [ ]:
# function to generate base policy
def base_policy_generator(curr_condition,curr_loc, b):
  base_policy = {}
  total_cost=0
  t=0
  while(check(curr_condition)):
    print(curr_condition)
    print(curr_loc)
    if(curr_condition[curr_loc]>0):
      y = y = feature_state_estimator(curr_loc,curr_condition[curr_loc],b)
      b[curr_loc] = belief_state_estimator(b[curr_loc],0,curr_condition[curr_loc])
      base_policy.update({tuple(y):0})
      curr_condition,curr_loc, c = one_time_step(curr_condition,curr_loc,0)
      total_cost += c*(1.01**t)
    else:
      i = curr_loc-1
      j = curr_loc+1
      while(i>=0 and j<10):
        if(curr_condition[i]>0):
          y = feature_state_estimator(curr_loc,curr_condition[curr_loc],b)
          b[curr_loc] = belief_state_estimator(b[curr_loc],-1,curr_condition[curr_loc])
          base_policy.update({tuple(y):-1})
          curr_condition,curr_loc, c = one_time_step(curr_condition,curr_loc,-1)
          total_cost += c*(1.01**t)
          break
        elif(curr_condition[j]>0):
          y = feature_state_estimator(curr_loc,curr_condition[curr_loc],b)
          b[curr_loc] = belief_state_estimator(b[curr_loc],1,curr_condition[curr_loc])
          base_policy.update({tuple(y):1})
          curr_condition,curr_loc, c = one_time_step(curr_condition,curr_loc,1)
          total_cost += c*(1.01**t)
          break
        i-=1
        j+=1
      if(i==-1):
        while(j<10):
          if(condition[j]>0):
            y = feature_state_estimator(curr_loc,curr_condition[curr_loc],b)
            b[curr_loc] = belief_state_estimator(b[curr_loc],1,curr_condition[curr_loc])
            base_policy.update({tuple(y):1})
            curr_condition,curr_loc, c = one_time_step(curr_condition,curr_loc,1)
            total_cost += c*(1.01**t)
            break
          j+=1
      elif(j==10):
        while(i>=0):
          if(curr_condition[i]>0):
            y = feature_state_estimator(curr_loc,curr_condition[curr_loc],b)
            b[curr_loc] = belief_state_estimator(b[curr_loc],-1,curr_condition[curr_loc])
            base_policy.update({tuple(y):-1})
            curr_condition,curr_loc, c = one_time_step(curr_condition,curr_loc,-1)
            total_cost += c
            break
          i-=1
    t+=1
  return base_policy, total_cost


In [ ]:
condition1 = copy.copy(condition) #copy of condition
print(condition1)

[0, 1, 0, 2, 1, 0, 1, 3, 2, 0]


In [ ]:
b_p, t_c = base_policy_generator(condition1,location,b1)
print(b_p)
print(t_c)

[0, 1, 0, 2, 1, 0, 1, 3, 2, 0]
7
[0, 1, 0, 2, 1, 0, 1, 0, 2, 0]
7
[0, 1, 0, 2, 1, 0, 1, 0, 2, 0]
6
[0, 1, 0, 2, 1, 0, 0, 0, 2, 0]
6
[0, 1, 0, 2, 1, 0, 0, 0, 2, 0]
5
[0, 1, 0, 2, 1, 0, 0, 0, 2, 0]
4
[0, 1, 0, 2, 0, 0, 0, 0, 2, 0]
4
[0, 1, 0, 2, 0, 0, 0, 0, 2, 0]
3
[0, 1, 0, 0, 0, 0, 0, 0, 2, 0]
3
[0, 2, 0, 0, 0, 0, 0, 0, 2, 0]
2
[0, 2, 0, 0, 0, 0, 0, 0, 2, 0]
1
[0, 0, 0, 0, 0, 0, 0, 0, 2, 0]
1
[0, 0, 0, 0, 0, 0, 0, 0, 2, 0]
2
[0, 0, 0, 0, 0, 0, 0, 0, 2, 0]
3
[0, 0, 0, 0, 0, 0, 0, 0, 2, 0]
4
[0, 0, 0, 0, 0, 0, 0, 0, 2, 0]
5
[0, 0, 0, 0, 0, 0, 0, 0, 2, 0]
6
[0, 0, 0, 0, 0, 0, 0, 0, 2, 0]
7
[0, 0, 0, 0, 0, 0, 0, 0, 2, 0]
8
{(0.7, 0.75, 0.8, 0.2, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0, 0.8, 0.2, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1, 0.1, 0.8, 0.1, 0.0, 0.8, 0.2, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0, 0.0, 0.0, 0.2, 0.8, 0.0, 0.1, 0.8, 0.1, 0.8, 0.2, 0.0, 0.0): 0, (0.7, 0.0, 0.8, 0.2, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0, 0.8, 0.2, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1, 0.1, 0.8, 0.1, 0.0, 0.8, 0.2, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0, 1.0, 

In [ ]:
print(b)
print(b1)

[[0.8 0.2 0.  0. ]
 [0.1 0.8 0.1 0. ]
 [0.8 0.2 0.  0. ]
 [0.  0.1 0.8 0.1]
 [0.1 0.8 0.1 0. ]
 [0.8 0.2 0.  0. ]
 [0.1 0.8 0.1 0. ]
 [0.  0.  0.2 0.8]
 [0.  0.1 0.8 0.1]
 [0.8 0.2 0.  0. ]]
[[8.00000000e-01 2.00000000e-01 0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [6.31098105e-04 1.35824600e-04 1.60809619e-01 8.38423458e-01]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [6.31098105e-04 1.35824600e-04 1.60809619e-01 8.38423458e-01]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [8.00000000e-01 2.00000000e-01 0.00000000e+00 0.00000000e+00]]


**Rollout** we used one-step lookahead
and 10 Monte-Carlo simulated trajectories with the base policy
from each leaf of the lookahead tree

In [ ]:
# function to estimate state-action values using rollouts
def Rollout(curr_condition,curr_loc, b, a):
  sum_Q_k =0
  y_s = feature_state_estimator(curr_loc,curr_condition[curr_loc],b)
  b[curr_loc]=belief_state_estimator(b[curr_loc], a, curr_condition[curr_loc])
  for k in range(0,10):
    Q_k=0
    curr_condition_1 = copy.copy(curr_condition)
    curr_loc_1 = copy.copy(curr_loc)
    curr_condition_1, curr_loc_1, Q_k = one_time_step(curr_condition_1,curr_loc_1,a)
    while(check(curr_condition_1)):
      a1 = np.random.choice([-1,0,1]) #we have to actually use base policy
      curr_condition_1, curr_loc_1, c = one_time_step(curr_condition_1, curr_loc_1, a1)
      Q_k += c
    sum_Q_k += Q_k
  Q = sum_Q_k/10
  return Q, y_s

In [ ]:
condition_t = copy.copy(condition)
print(condition_t)
b_t = copy.copy(b)
print(b_t)

[0, 1, 0, 2, 1, 0, 1, 3, 2, 0]
[[0.8 0.2 0.  0. ]
 [0.1 0.8 0.1 0. ]
 [0.8 0.2 0.  0. ]
 [0.  0.1 0.8 0.1]
 [0.1 0.8 0.1 0. ]
 [0.8 0.2 0.  0. ]
 [0.1 0.8 0.1 0. ]
 [0.  0.  0.2 0.8]
 [0.  0.1 0.8 0.1]
 [0.8 0.2 0.  0. ]]


In [ ]:
qt , yt = Rollout(condition_t,7,b_t,0)
print(qt)
print(yt)

906.8200000000004
[0.7  0.75 0.8  0.2  0.   0.   0.1  0.8  0.1  0.   0.8  0.2  0.   0.
 0.   0.1  0.8  0.1  0.1  0.8  0.1  0.   0.8  0.2  0.   0.   0.1  0.8
 0.1  0.   0.   0.   0.2  0.8  0.   0.1  0.8  0.1  0.8  0.2  0.   0.  ]


**Approximate Policy Iteration**

In [ ]:
l = b_p.values()
print(l)

dict_values([0, -1, 0, -1, -1, 0, -1, 0, -1, -1, 0, 1, 1, 1, 1, 1, 1, 1, 0])


In [ ]:
from keras.losses import categorical_crossentropy
from keras.utils import to_categorical
# API
def API(base_policy, curr_condition, curr_loc, b):
  curr_condition_1 = copy.copy(curr_condition)
  curr_condition_2 = copy.copy(curr_condition)
  curr_loc_1 = copy.copy(curr_loc)
  curr_loc_2 = copy.copy(curr_loc)
  b1 = copy.copy(b)
  b2 = copy.copy(b)
  t_c =0
  curr_policy = []
  l = b_p.values()
  for k in l:
    curr_policy.append(k)
  #policy network
  policy_model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(42)),
    tf.keras.layers.Dense(264, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')])
  policy_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics="Accuracy")
  for itr in range(1,2):
    X_train = []
    Y_train = []
    while(check(curr_condition_1)):
      Q_a = []
      y_a = []
      op_a = 0
      for a in range(0,3):
        Q_k , y_s = Rollout(copy.copy(curr_condition_1),copy.copy(curr_loc_1), copy.copy(b1), a-1)
        Q_a.append(Q_k)
        y_a.append(list(y_s))
      # print(Q_a)
      op_a = Q_a.index(min(Q_a)) - 1
      y_new = np.array(op_a)
      curr_condition_1, curr_loc_1, c = one_time_step(curr_condition_1,curr_loc_1,op_a)
      b1[curr_loc_1] = belief_state_estimator(b1[curr_loc_1],op_a,curr_condition_1[curr_loc_1])
      # print(c)
      # print(curr_condition_1)
      X_train.append(y_a[op_a])
      Y_train.append(y_new)
      t_c += c
    X_train = np.array(X_train)
    print(X_train.shape)
    Y_cat = np.zeros((len(Y_train), 3))
    for i in range(len(Y_train)):
      if Y_train[i] == -1:
        Y_cat[i, 0] = 1
      elif Y_train[i] == 0:
        Y_cat[i, 1] = 1
      else:
        Y_cat[i, 2] = 1
    print(Y_cat.shape)
    print(t_c)
    policy_model.fit(X_train,Y_cat, epochs=50)
    t1 = policy_model.predict(X_train)
    print(t1)
    approx_policy=[]
    y_s = []
    t_c1=0
    it=1
    while(check(curr_condition_2) and it < 20):
      x_s = feature_state_estimator(curr_loc_2, curr_condition_2[curr_loc_2], b2)
      x_array = np.array(x_s)
      x_array1 = x_array.reshape((1,42))
      print(x_array1.shape)
      p_a1 = policy_model.predict(x_array1)
      a1 = np.argmax(p_a1)-1
      print(a1)
      curr_condition_2, curr_loc_2, c1 = one_time_step(curr_condition_2, curr_loc_2, a1)
      b2[curr_loc_2] = belief_state_estimator(b2[curr_loc_2],a1, curr_condition_2[curr_loc_2])
      print(curr_condition_2)
      print(curr_loc_2)
      approx_policy.append(a1)
      t_c1 += c1
      print(c1)
      it+=1
    curr_policy = approx_policy
    print(t_c1)
  return policy_model

In [ ]:
condition_api = copy.copy(condition)
b_p_api = copy.copy(b_p)
b_api = copy.copy(b)
print(condition_api)
print(b_p_api)
print(b_api)

[0, 1, 0, 2, 1, 0, 1, 3, 2, 0]
{(0.7, 0.75, 0.8, 0.2, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0, 0.8, 0.2, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1, 0.1, 0.8, 0.1, 0.0, 0.8, 0.2, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0, 0.0, 0.0, 0.2, 0.8, 0.0, 0.1, 0.8, 0.1, 0.8, 0.2, 0.0, 0.0): 0, (0.7, 0.0, 0.8, 0.2, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0, 0.8, 0.2, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1, 0.1, 0.8, 0.1, 0.0, 0.8, 0.2, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1, 0.8, 0.2, 0.0, 0.0): -1, (0.6, 0.25, 0.8, 0.2, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0, 0.8, 0.2, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1, 0.1, 0.8, 0.1, 0.0, 0.8, 0.2, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1, 0.8, 0.2, 0.0, 0.0): 0, (0.6, 0.0, 0.8, 0.2, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0, 0.8, 0.2, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1, 0.1, 0.8, 0.1, 0.0, 0.8, 0.2, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1, 0.8, 0.2, 0.0, 0.0): -1, (0.5, 0.0, 0.8, 0.2, 0.0, 0.0, 0.1, 0.8, 0.1, 0.0, 0.8, 0.2, 0.0, 0.0, 0.0, 0.1, 0.8, 0.1, 0.1, 0.8, 0.1

In [ ]:
m = API(b_p_api,condition_api,6,b_api)

(40, 42)
(40, 3)
525.0
Epoch 1/50
2/2 [==============================] - 1s 11ms/step - loss: 1.1062 - Accuracy: 0.4500
Epoch 2/50
2/2 [==============================] - 0s 13ms/step - loss: 1.0329 - Accuracy: 0.6250
Epoch 3/50
2/2 [==============================] - 0s 8ms/step - loss: 0.9949 - Accuracy: 0.6000
Epoch 4/50
2/2 [==============================] - 0s 8ms/step - loss: 0.9643 - Accuracy: 0.5500
Epoch 5/50
2/2 [==============================] - 0s 8ms/step - loss: 0.9390 - Accuracy: 0.5500
Epoch 6/50
2/2 [==============================] - 0s 8ms/step - loss: 0.9161 - Accuracy: 0.6000
Epoch 7/50
2/2 [==============================] - 0s 10ms/step - loss: 0.8922 - Accuracy: 0.6500
Epoch 8/50
2/2 [==============================] - 0s 7ms/step - loss: 0.8703 - Accuracy: 0.6500
Epoch 9/50
2/2 [==============================] - 0s 8ms/step - loss: 0.8596 - Accuracy: 0.6500
Epoch 10/50
2/2 [==============================] - 0s 8ms/step - loss: 0.8469 - Accuracy: 0.6000
Epoch 11/50
2

KeyboardInterrupt: ignored